In [26]:
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
import os
os.environ["HUGGING_FACE_KEY"]=os.getenv("HUGGING_FACE_KEY")

In [28]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [29]:
embeddings.embed_query("Hello AI")

[0.01908188685774803,
 -0.04838583618402481,
 -0.0033970330841839314,
 -0.015432494692504406,
 0.026915451511740685,
 -0.037899527698755264,
 0.03683774173259735,
 0.0024586953222751617,
 0.031139591708779335,
 -0.002974053379148245,
 0.05297260358929634,
 0.027224618941545486,
 -0.03135930001735687,
 -0.03521350398659706,
 0.012696987949311733,
 -0.03398113697767258,
 -0.005759889259934425,
 0.04882349446415901,
 -0.025584226474165916,
 -0.016702925786376,
 0.03510289266705513,
 -0.0072209034115076065,
 -0.011071284301578999,
 -0.03638654947280884,
 0.02125084027647972,
 0.018635960295796394,
 0.022123489528894424,
 -0.053073909133672714,
 -0.03174823150038719,
 0.03385208174586296,
 -0.03267931193113327,
 0.05955952778458595,
 -0.04031715542078018,
 -0.01112906914204359,
 0.04654634743928909,
 -0.05826421454548836,
 0.03411427512764931,
 0.019681427627801895,
 0.008696145378053188,
 -0.004346166737377644,
 0.0029152834322303534,
 -0.08902806788682938,
 -0.015501082874834538,
 -0.0179

In [30]:
len(embeddings.embed_query("Hello AI"))

768

In [31]:
from pinecone import Pinecone

In [32]:
import os
pinecone_api_key=os.getenv("PINECONE_API_KEY")

In [33]:
pc=Pinecone(api_key=pinecone_api_key)

In [34]:
from pinecone import ServerlessSpec
#Serverless: Server will be Managed by the cloud provider

In [35]:
index_name="agenticbatch2"

In [36]:
pc.has_index(index_name)

True

In [37]:
#creating a index
if not pc.has_index(index_name):
    pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws",region="us-east-1")    
)

In [38]:
pc.has_index(index_name)

True

In [39]:
#loading the index
index=pc.Index(index_name)

In [42]:
from langchain_pinecone import PineconeVectorStore

In [43]:
vector_store=PineconeVectorStore(index=index,embedding=embeddings)

In [44]:
results = vector_store.similarity_search("what is a langchain?")

In [51]:
#### End to end Example using PDF document
from langchain_community.document_loaders import PyPDFLoader

In [52]:
### loader
FILE_PATH=r"C:\Users\saxenanidhi\agentic_ai\11-5-2025\data\llama2.pdf"
loader=PyPDFLoader(FILE_PATH)

In [45]:
loader.load()

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\saxenanidhi\\agentic_ai\\11-5-2025\\data\\llama2.pdf', 'total_pages': 77, 'page': 0, 'page_label': '1'}, page_content='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Kor

In [46]:
pages=loader.load()

In [47]:
### chunking/splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter =RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

In [48]:
split_docs=splitter.split_documents(pages)


In [49]:
len(split_docs)

615

In [53]:
vector_store.add_documents(documents=split_docs)

['70e6d29a-26d7-456f-95b1-c2f80d5293c4',
 '14bab859-775a-4a32-a44b-c4cf6cd66506',
 '0e113a89-f15a-4ce9-9d74-415673791bcb',
 '2d33d761-3652-4327-ae66-e6541be8f00b',
 '8a7a333a-c080-44fe-83f8-8929a5f910f4',
 '86e648d8-c9c1-482e-9c98-c9cecd9edfdb',
 '023b41d1-ac44-43d7-8f75-835f33f75c96',
 '7d8b7c2b-24b8-4150-b8f4-472e95c8e557',
 '855a4a2a-f6aa-4cdd-a2f2-7986feedb664',
 'd5444661-9ef5-46f7-be19-b687b7694af7',
 'f1b5a116-021e-468a-a05c-4bfb9d052040',
 'a2635002-b186-4cb7-a675-2380ab5e1dfe',
 '31483753-0415-4435-8f3e-f6ee6181d5cb',
 'c8b9897f-5e90-480f-a388-8758a767a29c',
 'eafa5693-577a-46ac-84c0-6061d342c112',
 '14eaf703-362a-4d23-ab77-6e4c22d0f025',
 'f94a3801-2556-4275-a0a8-e114ef810658',
 '268ccdeb-7a5c-4ac4-9e3a-e786e814d5a4',
 '05c16371-7034-49e1-839a-9090fdf84a70',
 '20b2225f-c9e2-41b2-a3e5-6adda0e28e72',
 'b90b79b7-7b4d-493a-89a6-4cbd1273b12a',
 '854fac99-ea56-4055-9df3-c45902a0dd4f',
 '01868947-34f7-4112-b255-7f7f1cab5acb',
 '16ee2b2d-b983-4daf-b2d6-58ae5a7b7e37',
 'ae48e87c-212f-

In [54]:
### Retriever
retriever=vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7} #hyperparameter
)

In [55]:
retriever.invoke("langchain")

[Document(id='dd1e5ea9-e121-4129-a365-37642c59c9cb', metadata={'author': '', 'creationdate': '2023-07-20T00:30:36+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'page': 23.0, 'page_label': '24', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': 'C:\\Users\\saxenanidhi\\agentic_ai\\11-5-2025\\data\\llama2.pdf', 'subject': '', 'title': '', 'total_pages': 77.0, 'trapped': '/False'}, page_content='preference data to train a safety reward model (see Section 3.2.2), and also reuse the adversarial prompts to\nsample from the model during the RLHF stage.\nBetter Long-Tail Safety Robustness without Hurting HelpfulnessSafety is inherently a long-tail problem,\nwhere the challenge comes from a small number of very specific cases. We investigate the impact of Safety\nRLHFbytakingtwointermediate Llama 2-Chatcheckpoints—onewithoutadversarialpromptsinth

In [56]:
### LLM
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [57]:
### prompt
from langchain import hub
prompt=hub.pull("rlm/rag-prompt")

In [58]:
import pprint

In [59]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [60]:
### output parser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [61]:
def format_docs(docs):
    return"\n\n".join(doc.page_content for doc in docs)

In [62]:
rag_chain=(
    {"context":retriever|format_docs, "question":RunnablePassthrough()}
    |prompt
    |model
    |StrOutputParser()
)

In [63]:
rag_chain.invoke("what is llama model?")

"Llama 2 is a large language model developed by Meta AI.  It comes in various sizes (7B, 13B, and 70B parameters) and is available in pretrained and fine-tuned versions.  It's an autoregressive language model using an optimized transformer architecture."